# Sandbox
This notebook covers the main steps in deploying a model.  Experimentation with features is encouraged.  Can you increase the NDCG reported during model training?

## Setup Client

In [46]:
from ltr.client import ElasticClient
client = ElasticClient()

## Step 1 - Create a Feature Set

In [64]:
'''
  TASK:
  Experiment with this featureset, the judgments are all title searches so what features could be useful?
  
  Ideas:
    - Search other fields
    - Phrase matches
    - Fuzzy matches
'''

config = {
   "featureset": {
        "features": [
            {
                "name": "title_match",
                "params": ["keywords"],
                "template": {
                    "match": {
                        "title": "{{keywords}}"
                    }
                }
            }
        ]
    }
}

from ltr import setup
setup(client, config=config, featureset='sandbox')

Removed Default LTR feature store [Status: 200]
Initialize Default LTR feature store [Status: 200]
Create sandbox feature set [Status: 201]


## Step 2 - Log Features for Training

In [65]:
from ltr.log import judgments_to_training_set
trainingSet = judgments_to_training_set(client, 
                                        judgmentInFile='data/title_judgments.txt', 
                                        trainingOutFile='data/sandbox_judgments_train.txt', 
                                        featureSet='sandbox')

Recognizing 40 queries...
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for rambo (0/40)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for rocky (1/40)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for war games (2/40)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for crocodile dundee (3/40)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for matrix (4/40)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for contact (5/40)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for space jam (6/40)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for battlestar galactica (7/40)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for her (8/40)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for jobs (9/40)


## Step 3 - Train a Model

In [66]:
'''
  TASK:
  Experiment with the leafs and trees variables, how do they affect NGCG?
  Does a high leaf value increase your NDCG?  What could be the potential downfalls?
'''
from ltr import train
trainLog  = train(client,
                  trainingInFile='data/sandbox_judgments_train.txt',
                  metric2t='NDCG@10',
                  leafs=20,
                  trees=20,
                  featureSet='sandbox',
                  modelName='sandbox')

print()
print("Impact of each feature on the model")
for ftrId, impact in trainLog.impacts.items():
    print("{} - {}".format(ftrId, impact))
    
for roundDcg in trainLog.rounds:
    print(roundDcg)
    
print("Train NDCG@10 %s" % trainLog.rounds[-1])

Running java -jar data/RankyMcRankFace.jar -ranker 6 -metric2t NDCG@10 -tree 20 -leaf 20 -train data/sandbox_judgments_train.txt -save data/sandbox_model.txt 
DONE
Delete model sandbox: 404
Created Model sandbox [Status: 201]

Impact of each feature on the model
1 - 35493.99538433604
0.9124
0.9124
0.9124
0.9124
0.9124
0.9124
0.9102
0.9073
0.9073
0.9072
0.9075
0.9093
0.9099
0.9099
0.9099
0.9099
0.9099
0.9099
0.9107
0.9116
Train NDCG@10 0.9116


## Search

In [67]:
from ltr import search
search(client, "rambo", modelName='sandbox')

{"size": 5, "query": {"sltr": {"params": {"keywords": "rambo"}, "model": "sandbox"}}}
Searching tmdb - {'size': 5, 'query': [Status: 200]
Rambo 
2.269571 
2008 
['Action', 'Thriller'] 
When governments fail to act on behalf of captive missionaries, ex-Green Beret John James Rambo sets aside his peaceful existence along the Salween River in a war-torn region of Thailand to take action.  Although he's still haunted by violent memories of his time as a U.S. soldier during the Vietnam War, Rambo can hardly turn his back on the aid workers who so desperately need his help. 
---------------------------------------
Rambo III 
1.7274473 
1988 
['Action', 'Adventure', 'Thriller', 'War'] 
Combat has taken its toll on Rambo, but he's finally begun to find inner peace in a monastery. When Rambo's friend and mentor Col. Trautman asks for his help on a top secret mission to Afghanistan, Rambo declines but must reconsider when Trautman is captured. 
---------------------------------------
Rambo: Firs